## Imports ##

In [106]:
import pandas as pd
import torch
import numpy as np
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [107]:
#Loading Dataset
DATASET_PATH = "../Datasets/Covid_Cleaned.csv"
COVID_data = pd.read_csv(DATASET_PATH)

## Batch preperation ##

In [108]:
X = COVID_data.drop(columns=['CLASIFFICATION_FINAL'])  # features (21)
y = COVID_data['CLASIFFICATION_FINAL']                 # target (low / high risk of COVID-19)

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42) # 70% train, 30% test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42) # test -> 15% validation, 15% final test

In [109]:
# Creating Pytorch Tensors
X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.to_numpy(), dtype=torch.float32)

X_val_tensor = torch.tensor(X_val.to_numpy(), dtype=torch.float32)
y_val_tensor = torch.tensor(y_val.to_numpy(), dtype=torch.float32)

X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [110]:
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

In [111]:
batch_size = 32

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


## Model Training ##

In [112]:
import torch.nn as nn
import torch.optim as optim
from src.utils import OurModel


In [113]:
ml = OurModel()

In [114]:
# model configuration
config = nn.Sequential(
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 20),
    nn.Tanh(),
    nn.Linear(20, 20),
    nn.ReLU(),
    nn.Linear(20, 1),
)
ml.add_configuration(config)

In [115]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.RMSprop(ml.parameters(), lr=0.001)

In [116]:
#Model Training

epochs = range(20)
for ep in epochs:
    ml.train()
    sum_loss = 0
    for input, true in train_loader:
        y_pred = ml(input)
        loss = criterion(y_pred, true)
        optimizer.zero_grad() # Reseting gradient
        loss.backward()
        optimizer.step() # updating
        sum_loss += loss.item()


RuntimeError: expected scalar type Long but found Float